## Imports and function definitions

In [460]:
import neo4j

import csv

import math
import numpy as np
import pandas as pd

import psycopg2

In [461]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [462]:
session = driver.session(database="neo4j")

In [463]:
def my_get_node_list():
    "get a list of nodes in the current graph"
    
    query = "match (n) return n.name as name"
    
    result = session.run(query)
    
    node_list = []
    
    for r in result:
        node_list.append(r["name"])
        
    node_list = sorted(node_list)
    
    return node_list

In [464]:
def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

In [465]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [466]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [467]:
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")


In [468]:
def my_neo4j_create_node(station_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)
    

In [469]:
def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [470]:
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [471]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [472]:
cursor = connection.cursor()

In [473]:
my_neo4j_wipe_out_database()

## Constructing graph

In [474]:
connection.rollback()

query = """

select station
from stations
order by station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    
    my_neo4j_create_node('depart ' + station)
    my_neo4j_create_node('arrive ' + station)
    

In [475]:
connection.rollback()

query = """

select station, line
from lines
order by station, line

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    line = row[1]
    
    depart = 'depart ' + station
    arrive = 'arrive ' + station
    line_station = line + ' ' + station
    
    my_neo4j_create_node(line_station)
    my_neo4j_create_relationship_one_way(depart, line_station, 0)
    my_neo4j_create_relationship_one_way(line_station, arrive, 0)
    

In [476]:
connection.rollback()

query = """

select a.station, a.line as from_line, b.line as to_line, s.transfer_time
from lines a
     join lines b
       on a.station = b.station and a.line <> b.line 
     join stations s
       on a.station = s.station
order by 1, 2, 3

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    from_line = row[1]
    to_line = row[2]
    transfer_time = int(row[3])
    
    from_station = from_line + ' ' + station
    to_station = to_line + ' ' + station
    
    my_neo4j_create_relationship_one_way(from_station, to_station, transfer_time)
    

In [477]:
connection.rollback()

query = """

select a.line, a.station as from_station, b.station as to_station, t.travel_time
from lines a
  join lines b
    on a.line = b.line and b.sequence = (a.sequence + 1)
  join travel_times t
    on (a.station = t.station_1 and b.station = t.station_2)
        or (a.station = t.station_2 and b.station = t.station_1)
order by line, from_station, to_station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    line = row[0]
    from_station = line + ' ' + row[1]
    to_station = line + ' ' + row[2]
    travel_time = int(row[3])
    
    my_neo4j_create_relationship_two_way(from_station, to_station, travel_time)
    

In [478]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 652
-------------------------


## Prepping table for features

In [479]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = """

CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})
"""

session.run(query)

In [480]:
connection.rollback()

query = """

drop table if exists graphy_features
;

create table graphy_features(
    node varchar(32),
    community numeric(5),
    pagerank float(16)
)
;

"""

cursor.execute(query)

connection.commit()

In [481]:
connection.rollback()

query = """

insert into graphy_features
values
(%s, 0, 0)
;

"""

node_list = my_get_node_list()

for node in node_list:
    cursor.execute(query, (node,))

connection.commit()

In [482]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select * 
from graphy_features
order by node

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,node,community,pagerank
0,arrive 12th Street,0,0
1,arrive 16th Street Mission,0,0
2,arrive 19th Street,0,0
3,arrive 24th Street Mission,0,0
4,arrive Antioch,0,0
...,...,...,...
209,yellow San Bruno,0,0
210,yellow SFO,0,0
211,yellow South San Francisco,0,0
212,yellow Walnut Creek,0,0


## Louvain Community Detection

In [483]:
query = """

CALL gds.louvain.stream('ds_graph', {includeIntermediateCommunities: true})
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).name AS name, communityId as community, intermediateCommunityIds as intermediate_community
ORDER BY community, name ASC

"""

result = session.run(query)

for r in result:
    
    query = "update graphy_features set community = %s where node = %s"
    
    cursor.execute(query, (r["community"], r["name"]))

connection.commit()

## Page Rank

In [484]:
query = """

CALL gds.pageRank.stream('ds_graph',
                         { maxIterations: $max_iterations,
                           dampingFactor: $damping_factor}
                         )
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score as pageRank
ORDER BY pageRank DESC, name ASC

"""

max_iterations = 20 # default 20
damping_factor = 0.85 # default 0.85

#my_neo4j_run_query_pandas(query, max_iterations=max_iterations, damping_factor=damping_factor)

result = session.run(query, max_iterations=max_iterations, damping_factor=damping_factor)

for r in result:
    
    query = "update graphy_features set pageRank = %s where node = %s"
    
    cursor.execute(query, (r["pageRank"], r["name"]))

connection.commit()

In [485]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select * 
from graphy_features
order by node

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

In [486]:
df.sort_values(['community', 'pagerank'])

,node,community,pagerank
69,depart 16th Street Mission,7,0.150000
71,depart 24th Street Mission,7,0.150000
1,arrive 16th Street Mission,7,0.502965
3,arrive 24th Street Mission,7,0.503332
50,blue 16th Street Mission,7,0.623158
...,...,...,...
151,orange Fremont,91,0.590169
140,green Warm Springs,91,0.593410
162,orange Warm Springs,91,0.593410
134,green Milpitas,91,0.603177


In [487]:
len(df['community'].unique())

11

## Checking logic with multiple station nodes

In [488]:
df[df['node'].str.contains('16th Street Mission')]

,node,community,pagerank
1,arrive 16th Street Mission,7,0.502965
50,blue 16th Street Mission,7,0.623158
69,depart 16th Street Mission,7,0.150000
120,green 16th Street Mission,7,0.623158
164,red 16th Street Mission,7,0.623159
188,yellow 16th Street Mission,7,0.623159


- we can see that community is all the same for each station
- page rank is also the same except arrive and depart nodes
- following suggestions doc, we will just ignore those nodes

In [489]:
df = df[df['node'].str.contains('arrive') == False]
df = df[df['node'].str.contains('depart') == False]
df[df['node'].str.contains('16th Street Mission')]

,node,community,pagerank
50,blue 16th Street Mission,7,0.623158
120,green 16th Street Mission,7,0.623158
164,red 16th Street Mission,7,0.623159
188,yellow 16th Street Mission,7,0.623159


In [490]:
df[df['node'].str.contains('SFO')]

,node,community,pagerank
184,red SFO,85,0.508938
210,yellow SFO,85,0.487344


- Looks like some pagerank values for different lines are still different
- Suggestions doc suggests to just average these, so we will do that in the future

## Consolidating nodes

In [491]:
new_df_dict = {'node': [], 'community': [], 'pagerank': []}
current_station = ' '
current_community = 0
current_pagerank = []
for i, row in df.iterrows():
    station = row['node']
    this_station = ' '.join(station.split(' ')[1:])
    if this_station != current_station: # we are on a new station, need to set values for the last one
        if current_station != ' ': # this if is so the 0 ghost community is not added
            new_df_dict['node'].append(current_station)
            new_df_dict['community'].append(current_community)
            new_df_dict['pagerank'].append(np.mean(current_pagerank))
        current_station = this_station
        current_community = row['community'] # we are assuming the community will be the same for all lines in one station based off quick visual inspection
        current_pagerank = [row['pagerank']]
    else:
        current_pagerank.append(row['pagerank'])
new_df = pd.DataFrame().from_dict(new_df_dict)
new_df

,node,community,pagerank
0,16th Street Mission,7,0.623158
1,24th Street Mission,7,0.623798
2,Balboa Park,29,0.635425
3,Bay Fair,47,0.675695
4,Castro Valley,47,0.571916
...,...,...,...
108,Rockridge,63,0.567313
109,San Bruno,85,0.608985
110,SFO,85,0.487344
111,South San Francisco,85,0.585070


## Finding delivery hubs

In [492]:
new_df.groupby(['community', 'node']).mean()

pagerank
community node                          
7         16th Street Mission   0.623159
          24th Street Mission   0.623803
21        Civic Center          0.623060
          Powell Street         0.623307
29        Balboa Park           0.635737
          Daly City             0.615695
          Glen Park             0.626299
31        Ashby                 0.567007
          Downtown Berkeley     0.583265
          El Cerrito Plaza      0.593005
          El Cerrito del Norte  0.603054
          North Berkeley        0.588795
          Richmond              0.477049
47        Bay Fair              0.651456
          Castro Valley         0.571916
          Dublin                0.457066
          Hayward               0.566689
          San Leandro           0.607288
          South Hayward         0.582948
          West Dublin           0.633782
51        Coliseum              0.676688
          Fruitvale             0.590684
          Lake Merritt          0.587100
          OAK                   0.386077
59        Embarcadero           0.628583
          Montgomery Street     0.624403
          West Oakland          0.644236
63        Antioch               0.463809
          Concord               0.640679
          Lafayette             0.633328
          North Concord         0.641970
          Orinda                0.617667
          Pittsburg             0.645698
          Pittsburg Center      0.657564
          Pleasant Hill         0.639852
          Rockridge             0.567313
          Walnut Creek          0.638228
67        12th Street           0.592356
          19th Street           0.612071
          MacArthur             0.652098
85        Colma                 0.550855
          Millbrae              0.554540
          SFO                   0.498141
          San Bruno             0.617093
          South San Francisco   0.586537
91        Berryessa             0.477086
          Fremont               0.590169
          Milpitas              0.603177
          Union City            0.587942
          Warm Springs          0.593410

So our delivery hubs are: <br>
    &emsp;- 24th Street Mission <br>
    &emsp;- Powell Street <br>
    &emsp;- Balboa Park <br>
    &emsp;- El Cerrito del Norte <br>
    &emsp;- Bay Fair <br>
    &emsp;- Coliseum <br>
    &emsp;- West Oakland <br>
    &emsp;- Pittsburg Center <br>
    &emsp;- MacArthur <br>
    &emsp;- San Bruno <br>
    &emsp;- Milpitas <br>

## Shortest path to delivery hubs from "kitchen"